### CHAPITRE 3 Homework 1

pubmed_cleaned.zip is a file containing a metadatas sample from pubmed articles. Your goal is to **convert the json to a mongo DB** and answer the following questions **USING** mongodb querys:

1) Create an index, explain your choice of key.

2) Delete every paper that was published prior 2019

3) How many paper have a single author ? Two authors ?

4) What's the last paper inserted in the db ?

5) Find articles with null meshwords.

6) Choose a keyword you are interested in (machine learning, computer vision,...). Find the number of articles with the choosen keyword in their meshwords, abstract or title.

7) What's the number of articles that have atleast one affiliation AND meshwords.

8) How many articles have a publishing date after 2020 ?

9) Find articles where there's atleast one affiliation from a choosen country (you decide which one).

10) Check for any duplicates. (hint: look at the doi or the pmid)

11) Remove every articles where the abstract starts with an "R".

12) Return the list of papers (pmid) where there's atleast one affiliation per author

13) Create 500 random samples of the dataset, compute a statistics that you are interested in and check how it behaves through the different samples

14) Sandbox exercise: think of a problematic and try to answer it.

In [ ]:
import json
import pymongo

database_client = pymongo.MongoClient('localhost', 27017)
# créer la base de données devoirs
my_database = database_client["homework"]
# créer la collection articles
articles_collection = my_database["pubmed"]
with open('C:/Users/epcmic/OneDrive/Bureau/pubmed_cleaned.zip') as file:
    file_data = json.load(file)

In [32]:
from bson import ObjectId

In [ ]:
# convertir les identifiants en ObjectId
for document in file_data:
    if '_id' in document and '$oid' in document['_id']:
        document['_id'] = ObjectId(document['_id']['$oid'])

# insérer les données dans la collection
articles_collection.insert_many(file_data)

In [ ]:
# récupérer le premier document de la collection
document = articles_collection.find_one()
print(document)

In [ ]:
from datetime import datetime

# supprimer tous les articles publiés avant 2019
date_before = datetime(2019, 1, 1, 0, 0)
result = articles_collection.delete_many({"date": {"$lt": date_before}})
print(f"Nombre de documents supprimés: {result.deleted_count}")


In [ ]:
# compter le nombre d'articles avec un seul auteur
num_single_author_papers = articles_collection.count_documents({"authors": {"$regex": "name ml"}})
print(f"Nombre de documents avec un seul auteur : {num_single_author_papers}")

In [ ]:
# Initialiser des compteurs pour les documents avec un auteur et deux auteurs
num_single_author_documents = 0
num_two_authors_documents = 0

# Compter le nombre de documents avec un seul auteur et deux auteurs
for document in articles_collection.find():
    authors = document['authors'].split(', ')
    num_authors = len(authors)
    if num_authors == 1:
        num_single_author_documents += 1
    elif num_authors == 2:
        num_two_authors_documents += 1

# Afficher les résultats
print("Nombre de documents avec un seul auteur :", num_single_author_documents)
print("Nombre de documents avec deux auteurs :", num_two_authors_documents)

In [ ]:
# Trouver le dernier document inséré dans la collection
last_paper = articles_collection.find_one(sort=[('_id', -1)])

# Afficher les détails du dernier document
print("Dernier document inséré :")
print("Titre :", last_paper['title'])
print("Auteurs :", last_paper['authors'])
print("Date :", last_paper['date'])

In [ ]:
from pymongo import MongoClient

# Se connecter à la base de données et sélectionner la collection
client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']

# Recherche des articles avec des meshwords nuls ou vides
query = {"meshwords": {"$in": [None, ""]}}
articles_with_null_meshwords = collection.find(query)

# Affichage des résultats de la recherche
print("Articles with null meshwords:")
for article in articles_with_null_meshwords:
    print("Title:", article['title'])

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['homework']
collection = db['pubmed']
keyword = "nuclear medicine" # On change le mot-clé d'intérêt

# Compter le nombre d'articles contenant le mot-clé dans les meshwords, le titre ou l'abstract
nombre_articles = articles_collection.count_documents({
    "$or": [
        {"meshwords": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"abstract": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"title": {"$regex": f".*{keyword}.*", "$options": "i"}}
    ]
})

print(f"Nombre d'articles avec le mot-clé '{keyword}': {nombre_articles}")

In [ ]:
article_count = articles_collection.count_documents({
    "$and": [
        {"affiliations": {"$exists": True}},
        {"meshwords": {"$exists": True}}
    ]
})

print(f"Nombre d'articles avec au moins une affiliation et des meshwords: {article_count}")

In [ ]:
# Count the number of articles with a publishing date after 2020
article_count = articles_collection.count_documents({
    "date": {"$gt": "year 2020"}
})

print(f"Number of articles with a publishing date after 2020: {article_count}")

In [ ]:
article_count = articles_collection.count_documents({
    "authors": {"$regex": "China"}})
print("Number of articles with at least one affiliation from China:", article_count)
article_country= articles_collection.find({"authors":{"$regex": "China"}})
for article in article_country:
    print("ID:", article["_id"],"Title :", article["title"])

In [ ]:
# Vérifier s'il y a des doublons

distinct_pmids = articles_collection.distinct("pmid")

# Vérifier s'il y a des doublons
duplicates = len(distinct_pmids) != articles_collection.count_documents({})

# Afficher le résultat
if duplicates:
    print("Des doublons ont été trouvés.")
else:
    print("Aucun doublon n'a été trouvé.")


In [ ]:
# Trouver les articles dont le résumé commence par la lettre "R"
articles_to_remove = list(articles_collection.find({"abstract": {"$regex": "^R"}}))

# Supprimer chaque article de la collection
for article in articles_to_remove:
    articles_collection.delete_one({"_id": article["_id"]})

# Vérifier le nombre d'articles restants dans la collection
article_count = articles_collection.count_documents({})
print(f"{article_count} articles remaining in collection.")

In [ ]:
articles_filtres = articles_collection.find({"abstract": {"$regex": "^\\s*\"R"}})
for article in articles_filtres:
    print("title :", article["title"])
    print("abstract :", article["abstract"])

In [ ]:
print("init count: ",articles_collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))
articles_collection.delete_many({"abstract": {"$regex": "^\\s*\"R"}})
print("after delete many: ",articles_collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))

In [ ]:
#Retourne la liste des articles (pmid) où chaque auteur a au moins une affiliation
articles = articles_collection.find(
    {"authors": {"$exists": True}}
)
liste_articles = [article["pmid"] for article in articles if len(article.get("authors", [])) > 0]
print(liste_articles)

In [ ]:
#Créer 500 échantillons aléatoires du jeu de données
#calculer une statistique qui vous intéresse et vérifier son comportement à travers les différents échantillons
#Définir le nombre d'échantillons
num_samples = 500

#Définir la statistique d'intérêt
statistics = []

#Échantillonner et calculer la statistique pour chaque échantillon
for i in range(num_samples):
    #Échantillonner des documents aléatoirement à partir de la collection
    sample = articles_collection.aggregate([
        {"$sample": {"size": 100}}  #Ajuster la taille de l'échantillon au besoin
    ])

    # Calculer la statistique d'intérêt pour l'échantillon
    # Ici, nous calculons la taille d'équipe moyenne
    team_sizes = [doc.get("team_size", 0) for doc in sample]
    average_team_size = sum(team_sizes) / len(team_sizes)
    
    # Ajouter la statistique calculée à la liste
    statistics.append(average_team_size)

#Afficher les statistiques
print(statistics)